In [0]:
dbutils.widgets.text("uc_catalog", "main")
dbutils.widgets.text("uc_schema", "mfg_agent_bricks_demo")
dbutils.widgets.text("uc_volume", "mfg_agent_bricks_demo_unstructured_data")

UC_CATALOG = dbutils.widgets.get("uc_catalog")
UC_SCHEMA = dbutils.widgets.get("uc_schema")
UC_VOLUME = dbutils.widgets.get("uc_volume")

### Demand vs. Supply reconciliation

In [0]:
%sql
-- Projected ending on hand (EOH) and stockout risk
WITH demand AS (
  SELECT dc_id, sku, demand_date AS dt, SUM(forecast_units) AS fc_units
  FROM main.mfg_agent_bricks_demo.demand_forecast_daily
  GROUP BY dc_id, sku, demand_date
), inbound AS (
  SELECT dc_id, sku, eta_date AS dt, SUM(inbound_units) AS inbound_units
  FROM main.mfg_agent_bricks_demo.supply_plan_inbound
  GROUP BY dc_id, sku, eta_date
), inv AS (
  SELECT dc_id, sku, as_of_date AS dt, on_hand_units, safety_stock
  FROM main.mfg_agent_bricks_demo.inventory_positions
), grid AS (
  SELECT i.dc_id, i.sku, i.dt, i.on_hand_units, i.safety_stock,
         COALESCE(d.fc_units,0) AS fc_units, COALESCE(s.inbound_units,0) AS inbound_units
  FROM inv i
  LEFT JOIN demand d ON d.dc_id=i.dc_id AND d.sku=i.sku AND d.dt=i.dt
  LEFT JOIN inbound s ON s.dc_id=i.dc_id AND s.sku=i.sku AND s.dt=i.dt
), proj AS (
  SELECT dc_id, sku, dt,
         SUM(inbound_units - fc_units) OVER (PARTITION BY dc_id, sku ORDER BY dt ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
         + FIRST_VALUE(on_hand_units) OVER (PARTITION BY dc_id, sku ORDER BY dt) AS projected_eoh,
         safety_stock
  FROM grid
)
SELECT dc_id, sku, dt AS as_of_date, projected_eoh, safety_stock,
       CASE WHEN projected_eoh < safety_stock THEN TRUE ELSE FALSE END AS at_risk
FROM proj
QUALIFY ROW_NUMBER() OVER (PARTITION BY dc_id, sku ORDER BY as_of_date DESC)=1
ORDER BY projected_eoh ASC, dc_id, sku;


### mfg_agent_bricks_demo revenue and margin across suppliers, distributors, timeframe

In [0]:
%sql
WITH so AS (
  SELECT date_trunc('month', order_date) AS month, sku, distributor_id, region,
         SUM(units) AS units, SUM(units*unit_price) AS revenue
  FROM main.mfg_agent_bricks_demo.sales_orders
  GROUP BY 1, 2, 3, 4
),
cogs AS (
  SELECT sku, MAX_BY(unit_cogs, effective_date) AS unit_cogs
  FROM main.mfg_agent_bricks_demo.cogs_reference
  GROUP BY sku
),
supply_link AS (
  -- demo heuristic: dominant inbound supplier per SKU
  SELECT sku, supplier_id
  FROM (
    SELECT sku, supplier_id, ROW_NUMBER() OVER (PARTITION BY sku ORDER BY COUNT(*) DESC) AS r
    FROM main.mfg_agent_bricks_demo.supply_plan_inbound
    GROUP BY sku, supplier_id
  ) WHERE r=1
)
SELECT so.month, so.sku, sl.supplier_id, so.distributor_id, so.region,
       so.revenue,
       so.units * c.unit_cogs AS cogs,
       so.revenue - (so.units * c.unit_cogs) AS margin,
       CASE WHEN so.revenue>0 THEN (so.revenue - so.units*c.unit_cogs)/so.revenue ELSE NULL END AS margin_pct
FROM so
JOIN cogs c USING (sku)
LEFT JOIN supply_link sl USING (sku)
WHERE
  (COALESCE(:sku, '') = '' OR so.sku = :sku)
  AND (COALESCE(:region, '') = '' OR so.region = :region)
ORDER BY so.month, so.sku, so.distributor_id, so.region;


### Supplier SLA snapshot

In [0]:
%sql
-- Supplier SLA snapshot for narrative responses
SELECT s.supplier_id, s.supplier_name, d.title, d.format, d.effective_date, d.expiry_date, d.text
FROM main.mfg_agent_bricks_demo.suppliers s
JOIN main.mfg_agent_bricks_demo.contract_texts d
  ON d.entity_type='supplier' AND d.entity_id = s.supplier_id
ORDER BY s.supplier_id;


### Distributor terms snapshot

In [0]:
%sql 
-- Distributor terms snapshot
SELECT dist.distributor_id, dist.distributor_name, d.title, d.format, d.effective_date, d.expiry_date, d.text
FROM main.mfg_agent_bricks_demo.distributors dist
JOIN main.mfg_agent_bricks_demo.contract_texts d
  ON d.entity_type='distributor' AND d.entity_id = dist.distributor_id
ORDER BY dist.distributor_id;

### Finance queries

What are monthly units and revenue by SKU and distributor

In [0]:
%sql
SELECT
  date_trunc('month', order_date) AS month,
  sku,
  distributor_id,
  SUM(units) AS units,
  SUM(units * unit_price) AS revenue
FROM main.mfg_agent_bricks_demo.sales_orders
GROUP BY date_trunc('month', order_date), sku, distributor_id
ORDER BY month, sku, distributor_id;